In [1]:
import sys
print("python version: ", sys.version)

import pandas as pd
print("pandas version: ", pd.__version__)

import numpy as np
print('numpy version: ', np.__version__)

import sklearn
print('sklearn version: ', sklearn.__version__)

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import keras
print('keras version: ', keras.__version__)
print('keras backend: ', keras.backend.backend())

import tensorflow as tf
print('tensorflow version: ', tf.__version__)

import pickle

import random

from numpy.random import seed
from tensorflow import set_random_seed

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, auc,\
                            roc_curve, precision_recall_curve
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, GRU, Dense, Dropout, Masking, TimeDistributed
from keras.optimizers import Adagrad, Adadelta, SGD
from keras.preprocessing.sequence import pad_sequences

python version:  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
pandas version:  0.24.2
numpy version:  1.16.4
sklearn version:  0.21.2


Using TensorFlow backend.


keras version:  2.2.4
keras backend:  tensorflow
tensorflow version:  1.14.0


In [2]:
EXPERIMENT_NO = 1

In [3]:
file = open('processed_data/experiment%d_padded_data' % EXPERIMENT_NO, 'rb')
X_train_pad, y_train_array, X_test_pad, y_test_array = pickle.load(file)
file.close()

In [4]:
file = open('processed_data/experiment%d_data' % EXPERIMENT_NO, 'rb')
X_train, y_train_array, X_test, y_test_array = pickle.load(file)
file.close()

In [5]:
model = keras.models.load_model('trained_models/experiment%d_model%d.h5' % (EXPERIMENT_NO, 0))

W1026 12:24:14.405682 140555928008512 deprecation_wrapper.py:119] From /usr/local/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1026 12:24:14.439850 140555928008512 deprecation_wrapper.py:119] From /usr/local/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1026 12:24:14.441454 140555928008512 deprecation_wrapper.py:119] From /usr/local/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1026 12:24:14.554350 140555928008512 deprecation_wrapper.py:119] From /usr/local/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

In [9]:
# model.predict(X_test[0])
X_test.values[0]

array([1.03790000e+04, 7.00000000e+00, 7.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.40000000e+01, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.20000000e+01,
       2.33333333e+00, 0.00000000e+00, 2.10000000e+01, 2.10000000e+01,
       0.00000000e+00, 0.00000000e+00, 7.07106781e-01, 7.07106781e-01,
       7.07106781e-01, 7.07106781e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [6]:
def make_prediction(model, X, generator=None):
    if(generator is not None):
        y_predict = model.predict_generator(generator(X), steps=len(X))
    else:
        y_predict = model.predict(X)
    return np.concatenate(y_predict)

def evaluation(model, X, y, generator=None):
    y_predict = make_prediction(model, X, generator=generator)
    f1, th = find_best_f1(y_predict, y)
    prec, recall, accu, spec = other_metrics(y_predict, y, th)
    return { 'auroc': auroc(y_predict, y), 'f1': f1, 'threshold': th, 
            'precision': prec, 'recall': recall, 'accuracy': accu, 'specificity': spec }

# calculate evaluation metrics
def auroc(y_predict, y_true, plot=False):
    fpr, tpr, thresholds = roc_curve(y_true, y_predict, pos_label=1)    
    if(plot):
        plot_roc_curve(fpr, tpr)
    return auc(fpr, tpr)

def find_best_f1(y_predict, y_true):
    fpr, tpr, thresholds = roc_curve(y_true, y_predict, pos_label=1) 
    max_f1, th = 0, 0
    for threshold in thresholds:
        f1 = f1_score(y_true, y_predict > threshold)
        if(f1 > max_f1):
            max_f1 = f1
            th = threshold
    return max_f1, th

def other_metrics(y_predict, y_true, threshold):
    y_threshold = y_predict > threshold
    tn, fp, fn, tp = confusion_matrix(y_true, y_threshold).ravel()
    return precision_score(y_true, y_threshold), recall_score(y_true, y_threshold),\
            accuracy_score(y_true, y_threshold), float(tn) / (tn + fp)

# draw roc plot
def plot_roc_curve(fpr, tpr):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.show()

In [7]:
evaluation(model, X_test_pad, y_test_array)

{'auroc': 0.7271231508644903,
 'f1': 0.44644644644644643,
 'threshold': 0.20309684,
 'precision': 0.39399293286219084,
 'recall': 0.5150115473441108,
 'accuracy': 0.7577748576434516,
 'specificity': 0.8145945945945946}